### CDS NYU
### DS-GA 3001 | Reinforcement Learning
### Lab 06
### March 06, 2025


# Implement RL algorithms with Keras-RL or Stable-Baseline

<br>

---

## Section Leader


Akshitha Kumbam – ak11071@nyu.edu

Kushagra Khatwani – kk5395@nyu.edu


## Goal of Today's Lab 

In this Lab, we will implement RL algorithms by building on existing RL algorithm libraries, so we don't have to implement RL agents from scratch as we did in the past few weeks. By doing so, we get less control on the details of the implementation, but it is much faster to implement (**we will cover 4 case studies today:** `CartPole`, `SpaceInvaders`, `CarRacing`, and `StockTrading`), and components available in public libraries tend to be high quality and efficient. 

Let us start with the DQN algorithm. We can use external open-source Python packages which implement each of the key DQN methods (e,g., experience replay method, action-selection method, etc). These methods are developed, maintained, and optimized for robustness to different scenarios, and for overall performance.

Using these packages instead of implementing each DQN component from scratch is generally faster, leads to a more reliable/efficient program, yet still gives you a lot of control on the details of the implementation and hyperparameters.

We will focus on DQN in the first part of the lab, but these packages also offer methods for most commonly used RL algorithms such as A3C, PPO, etc, so we will start looking at these too.

## Resources

* https://gymnasium.farama.org/


# 1. Implement DQN with Keras-RL Methods

Keras-RL is an older reinforcement learning library that does not support the latest versions of TensorFlow and Gym. To ensure compatibility, we need to downgrade both TensorFlow and Gym to specific versions that work with Keras-RL.

### Install Required Dependencies

To install the required versions, run the following commands:

```bash
pip install tensorflow==2.15
pip install gym==0.15.4
pip install pyglet==1.5.11
```

**Note:** We use `gym` instead of `gymnasium` because Keras-RL was originally built for `gym` and does not support `gymnasium`. Using these specific versions ensures that we can successfully implement and train a Deep Q-Network (DQN) using Keras-RL.

In [3]:
!pip install tensorflow==2.15
!pip install gym==0.15.4
!pip install pyglet==1.5.11

  Using cached tensorflow-2.15.0-cp39-cp39-win_amd64.whl.metadata (3.6 kB)
  Using cached tensorflow_intel-2.15.0-cp39-cp39-win_amd64.whl.metadata (5.1 kB)
  Using cached tensorflow_estimator-2.15.0-py2.py3-none-any.whl.metadata (1.3 kB)
Using cached tensorflow-2.15.0-cp39-cp39-win_amd64.whl (2.1 kB)
Using cached tensorflow_intel-2.15.0-cp39-cp39-win_amd64.whl (300.8 MB)
Using cached tensorflow_estimator-2.15.0-py2.py3-none-any.whl (441 kB)
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 1.5/1.5 MB 15.7 MB/s eta 0:00:00



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached pyglet-1.3.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pyglet-1.3.2-py2.py3-none-any.whl (1.0 MB)
  Attempting uninstall: pyglet
    Found existing installation: pyglet 1.5.11
    Uninstalling pyglet-1.5.11:
      Successfully uninstalled pyglet-1.5.11



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached pyglet-1.5.11-py3-none-any.whl.metadata (7.6 kB)
Using cached pyglet-1.5.11-py3-none-any.whl (1.1 MB)
  Attempting uninstall: pyglet
    Found existing installation: pyglet 1.3.2
    Uninstalling pyglet-1.3.2:
      Successfully uninstalled pyglet-1.3.2


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gym 0.15.4 requires pyglet<=1.3.2,>=1.2.0, but you have pyglet 1.5.11 which is incompatible.

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
!pip install keras==2.10.0
!pip install keras-rl2

  Using cached keras-2.10.0-py2.py3-none-any.whl.metadata (1.3 kB)
Using cached keras-2.10.0-py2.py3-none-any.whl (1.7 MB)
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 2.10.0 which is incompatible.

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached keras-2.15.0-py3-none-any.whl.metadata (2.4 kB)
Using cached keras-2.15.0-py3-none-any.whl (1.7 MB)
  Attempting uninstall: keras
    Found existing installation: keras 2.10.0
    Uninstalling keras-2.10.0:
      Successfully uninstalled keras-2.10.0



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



# 1.1 Solve *Cart Pole*  with DQN from Keras-rl

Most of this case study will be the same as in previous lab, but we build some of the key components of the DQN agent using components available in the Keral-RL library


## Imports

In [2]:
!pip install pyglet

   ---------------------------------------- 0.0/962.1 kB ? eta -:--:--
   --------------------------------------- 962.1/962.1 kB 14.9 MB/s eta 0:00:00



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import time 
import gym
from pyglet.window import key 
from tensorflow.keras.models import Sequential  
from tensorflow.keras.layers import Dense  
from tensorflow.keras.layers import Activation 
from tensorflow.keras.layers import Flatten  
from tensorflow.keras.optimizers.legacy import Adam  # Adam optimizer


# Import DQN methods from the keras-rl2 library (keras-rl is tagged "rl" in Python)
# Quick fix if python cannot import name '__version__' from 'tensorflow.keras'
import tensorflow as tf
from keras import __version__
tf.keras.__version__ = __version__

from rl.agents.dqn import DQNAgent 

In [2]:
print(__version__)

2.15.0


## Set up the `CartPole` Gym environment

In [3]:
# https://stackoverflow.com/questions/56904270/difference-between-openai-gym-environments-cartpole-v0-and-cartpole-v1
env_name = ENV_NAME = 'CartPole-v1'
env = gym.make(env_name)  

# Same as last week:
num_actions = env.action_space.n
num_observations = env.observation_space.shape[0]
print(f"There are {num_actions} possible actions and {num_observations} observations")


There are 2 possible actions and 4 observations


## Execute random actions just to get familiar with the environment

In [4]:
# Load the CartPole Gym environment with graphical rendering to vizualize the environment
env_test = gym.make("CartPole-v1")  # [Jeremy] was v1 last week  
# Set to initial state
env_test.reset()
  

# Loop over 200 steps
for _ in range(200):
    env_test.render()                                                 # Render on the screen
    action = env_test.action_space.sample()                           # Choose a random action
    new_state, reward, done, info = env_test.step(action)  # Carry out the action
    time.sleep(0.03)
    if done:
         env_test.reset()
            
env_test.close()


## Implement an Artificial Neural Network
To build our network, we first need to find out how many actions and observation our environment has.
We can either get those information from the source code (https://github.com/openai/gym/blob/master/gym/envs/classic_control/cartpole.py) or via the following commands:

Similar to in the previous lab, we build a simple ANN with 2 hidden layers and 16 and 32 neurons each followed by relu activation. The output layer has 2 nodes, one for each action

In [5]:
model = Sequential()
# https://keras.io/api/layers/reshaping_layers/flatten/
model.add(Flatten(input_shape=(1,) + env.observation_space.shape))

model.add(Dense(16))
model.add(Activation('relu'))

model.add(Dense(32))
model.add(Activation('relu'))

model.add(Dense(num_actions))
model.add(Activation('linear'))

print(model.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 4)                 0         
                                                                 
 dense (Dense)               (None, 16)                80        
                                                                 
 activation (Activation)     (None, 16)                0         
                                                                 
 dense_1 (Dense)             (None, 32)                544       
                                                                 
 activation_1 (Activation)   (None, 32)                0         
                                                                 
 dense_2 (Dense)             (None, 2)                 66        
                                                                 
 activation_2 (Activation)   (None, 2)                 

## Implement the DQN framework with Keras-RL

The DQN agent `DQNagent` from Keras-RL takes the following parameters:

1. `model` = The ANN


2. `nb_actions` = The number of actions (2 in this case)


3. `memory` = The action replay memory. Ny far the most common choice is `SequentialMemory()` 


4. `nb_steps_warmup` = Number of iterations used to fill the memory prior starting to update the ANN parameters


5. `target_model_update` = How often (in number of steps) to update the target model


6. `policy` = You can choose between `LinearAnnealedPolicy()`, `SoftmaxPolicy()`, `EpsGreedyQPolicy()`, `GreedyQPolicy()`, `GreedyQPolicy()`, `MaxBoltzmannQPolicy()` and `BoltzmannGumbelQPolicy()`. We will use the `LinearAnnealedPolicy` policy, but feel free to try them out and inspect which works best here


There are some more parameters you can pass to the DQN Agent, feel free to explore them on your own.

let's initialize a circular buffer with a limit of 20000 and window length of 1 (window length describes the number of steps stored to define a state)


In [6]:
from rl.memory import SequentialMemory  # Sequential Memory for storing observations (optimized circular buffer)

memory = SequentialMemory(limit=20000, window_length=1)


Then we define the Action Selection Policy: <br />
We use *LinearAnnealedPolicy* in order to perform the epsilon greedy strategy with decaying epsilon. <br />
*LinearAnnealedPolicy* accepts an action selection policy, its maximal and minimal values and a step number in order to create a dynamical policy. <br/>
The smallest value epsilon can reach during training is 0.1.<br />
For testing/evaluation of the trained agent, let's set epsilon to 0.05


In [7]:
# LinearAnnealedPolicy allows to decay the epsilon for the epsilon greedy strategy
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), 
                              attr='eps',
                              value_max=1.,
                              value_min=.1,
                              value_test=.05,
                              nb_steps=20000) 


Now we create the DQN Agent based on the defined model (**model**), the possible actions (**num_actions**) (left and right in this case), the circular buffer (**memory**), the warmup phase (**10**), how often the target model gets updated (**100**) and the policy (**policy**)


In [8]:
dqn = DQNAgent(model = model, nb_actions = num_actions, memory = memory, nb_steps_warmup = 10,
               target_model_update = 100, policy = policy)



We can now compile the DQN with the Adam optimizer and a learning rate of 0.001.<br />
We log the Mean Absolute Error

In [9]:
# Use learning_rate instead of lr if you get warning
dqn.compile(Adam(learning_rate=1e-3), metrics=['mae']) 

Let's run the training for 20000 steps. You can change visualize=True if you want to watch your model learning.
Keep in mind that this increases the running time.


## Train the DQN agent

In [10]:
dqn.fit(env, nb_steps=20000, visualize=False, verbose=2)

Training for 20000 steps ...




f:\Study_Notes_Backup\Data_Science\DS-GA-3001-Reinforcement\.venv\lib\site-packages\keras\src\engine\training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
f:\Study_Notes_Backup\Data_Science\DS-GA-3001-Reinforcement\.venv\lib\site-packages\rl\memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


    21/20000: episode: 1, duration: 0.318s, episode steps:  21, steps per second:  66, episode reward: 21.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.476 [0.000, 1.000],  loss: 0.617167, mae: 0.616144, mean_q: 0.170667, mean_eps: 0.999302
    39/20000: episode: 2, duration: 0.056s, episode steps:  18, steps per second: 319, episode reward: 18.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.333 [0.000, 1.000],  loss: 0.462709, mae: 0.574850, mean_q: 0.248904, mean_eps: 0.998673
    75/20000: episode: 3, duration: 0.106s, episode steps:  36, steps per second: 339, episode reward: 36.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.556 [0.000, 1.000],  loss: 0.225440, mae: 0.568757, mean_q: 0.542542, mean_eps: 0.997458
    88/20000: episode: 4, duration: 0.041s, episode steps:  13, steps per second: 321, episode reward: 13.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.769 [0.000, 1.000],  loss: 0.059013, mae: 0.596585, mean_q: 0.943146, mean_ep

f:\Study_Notes_Backup\Data_Science\DS-GA-3001-Reinforcement\.venv\lib\site-packages\rl\memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


   116/20000: episode: 5, duration: 0.091s, episode steps:  28, steps per second: 309, episode reward: 28.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.571 [0.000, 1.000],  loss: 0.268794, mae: 0.889782, mean_q: 1.224093, mean_eps: 0.995432
   135/20000: episode: 6, duration: 0.054s, episode steps:  19, steps per second: 355, episode reward: 19.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.632 [0.000, 1.000],  loss: 0.152110, mae: 1.174691, mean_q: 2.032136, mean_eps: 0.994375
   178/20000: episode: 7, duration: 0.117s, episode steps:  43, steps per second: 367, episode reward: 43.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.558 [0.000, 1.000],  loss: 0.144724, mae: 1.185773, mean_q: 2.157195, mean_eps: 0.992980
   196/20000: episode: 8, duration: 0.052s, episode steps:  18, steps per second: 344, episode reward: 18.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.389 [0.000, 1.000],  loss: 0.087339, mae: 1.161379, mean_q: 2.226037, mean_ep

After just 1-2 minutes of training (for reference, it takes < 1min on a Macbook Air with M2), we achieve some great results already. It was taking at least 15 minutes of training to reach a similar level of performance with the custom DQN implemented from scratch in the previous lab.

The reason for this is that keras-rl has implemented many optimization strategies (e.g., the optimized replay buffer) which lead to a significantly faster convergence than the DQN we implemented by hand.

In [11]:
# After training is done, we can save the final weights.
dqn.save_weights(f'dqn_{env_name}_weights.h5f', overwrite=True)

## Exploit learned Q values in test simulations

The Keras-RL agents also offer methods to perform tests in Gym, with some parameters e.g. to decide whether to render the simulation graphically.

In [12]:
# Finally, evaluate our algorithm for 5 episodes.
dqn.test(env, nb_episodes=5, visualize=True)
env.close()

Testing for 5 episodes ...
Episode 1: reward: 500.000, steps: 500
Episode 2: reward: 365.000, steps: 365
Episode 3: reward: 399.000, steps: 399
Episode 4: reward: 333.000, steps: 333
Episode 5: reward: 344.000, steps: 344


All in all, we accomplished a better agent (trained more efficiently) with much less code than in the previous lab, thanks to Keras-RL!

---------

# 1.2 Solve *Space Invaders*  with Convolutional DQN from Keras-rl
Also, install :
```bash
pip install "gym[atari,accept-rom-license]"
```

## Imports

In [18]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras.optimizers.legacy import Adam

# Quick fix if python cannot import name '__version__' from 'tensorflow.keras'
import tensorflow as tf
from keras import __version__
tf.keras.__version__ = __version__

from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

import gym

env = gym.make('SpaceInvaders-v0')


## Execute random actions just to get familiar with the environment

In [ ]:
env = gym.make('SpaceInvaders-v0')

episodes = 5

for episode in range(1, episodes):
    state = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        state, reward, done, info = env.step(env.action_space.sample())
        print(state.shape)
        score += reward
    print('Episode: {}\nScore: {}'.format(episode, score))
    
env.close()

## Implement a Convolutional Neural Network

In [20]:
def build_model(height, width, channels, actions):
    model = Sequential()
    model.add(Conv2D(32, (8,8), strides=(4,4), activation='relu', input_shape=(3, height, width, channels)))
    model.add(Conv2D(64, (4,4), strides=(2,2), activation='relu'))
    model.add(Conv2D(64, (4,4), strides=(2,2), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [ ]:
height, width, channels = env.observation_space.shape
actions = env.action_space.n
print(height, width, channels)

In [ ]:
#del model # [Jeremy] why do we need this? TBD

In [22]:
model = build_model(height, width, channels, actions)

## Implement the DQN framework 

In [23]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=10000)
    memory = SequentialMemory(limit=2000, window_length=3)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  enable_dueling_network=False, dueling_type='avg',
                  nb_actions=actions, nb_steps_warmup=1000)
    return dqn

In [24]:
dqn = build_agent(model, actions)

In [ ]:
dqn.compile(Adam(learning_rate=0.001))

In [ ]:
dqn.fit(env, nb_steps = 1000, visualize = False, verbose = 1) # Train for 1000 steps in class, try 40000 at home :)

In [ ]:
#dqn.save_weights('SpaceInvaderTrainedModel/dqn.h5f')

## Exploit learned Q values in test simulations

In [ ]:
#dqn.load_weights('SpaceInvaderTrainedModel/dqn.h5f')

In [ ]:
env = gym.make('SpaceInvaders-v0') 

scores = dqn.test(env, nb_episodes = 10, visualize = True)  # Would need play with versioning to use the vizualize parameter (currently fixed with line above)
print(np.mean(scores.history['episode_reward']))

# 2. Implement DQN with Stable-baselines algorithms

Instead of implementing DQN from scratch (previous lab), or its individual components from external packages as seen in the section above, yet other packages exist which offer a "complete" implementation of the most popular RL algorithms (DQN, A3C, PPO, DDPG, etc).

Many such packages exist. The most popular ones include OpenAI `Stable-baselines`, DeepMind `Acme`, AWS `SageMaker RL`, Meta `AI ReAgent`, Ray `RLlib`, Intel `AI Coach`. 

Today we will focus exclusively on `Stable-baselines` because it was initially designed by OpenAI in tandem with `Gym` environments. In the final labs of the semester will introduce several other frameworks.


These implementations of RL algorithms have been optimized for ease-of-use, robustness to different scenarios, and overall performance. Their drawback is you get less control on details of the implementation and hyperparameters, compared to using packages such as `Keras-RL` which only implement key *components* of RL algorithms.

Please install the following :
```bash
pip install stable-baselines3
```

# 2.1 Play 'Car Racing' with Convolutional PPO from Stable-baselines

## Imports

In [5]:
import gymnasium as gym

#Quick fix for M1 architecture (M2/M3 might also need this).
# import os
# os.environ['KMP_DUPLICATE_LIB_OK']='True'

from stable_baselines3 import PPO 
from stable_baselines3.common.evaluation import evaluate_policy

## Set up the `CarRacing` Gym environment

In [6]:
env = gym.make('CarRacing-v3')
env.observation_space.sample().shape
env.action_space.sample()
#env.reset()
#env.render()
env.close()

In [14]:
!pip install shimmy>=2.0


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Execute random actions just to get familiar with the environment

In [ ]:
env_test = gym.make('CarRacing-v3', render_mode = 'human')
episodes = 5
for episode in range(episodes):
    state, info = env_test.reset()
    done = False
    score = 0
    
    while not done:
        env_test.render()
        action = env_test.action_space.sample()
        n_state, reward, done, trunc, info = env_test.step(action)
        score+=reward
    print("Episode: {} Score: {}".format(episode, score))
    
env.close()

In [50]:
# from stable_baselines3.common.env_util import make_vec_env
# env = make_vec_env(lambda: gym.make('CarRacing-v3'), n_envs=1)

## Train the `PPO` algorithm from `stable-baselines`

In [ ]:
model = FILL

In [ ]:
# Train the model
# TODO


## Exploit the trained agent in test simulations

In [ ]:
# Evaluate the policy
# TODO

In [6]:
env.close()

# 2.2 Trade a S&P 500 stock index with DQN from Stable-baselines
Please install the following:
```bash
pip install gym_anytrading
pip install yfinance  
pip install pandas_datareader
pip install TA
```


In [28]:
import gymnasium as gym

#Quick fix for M1 architecture (M2/M3 might also need this).
# import os
# os.environ['KMP_DUPLICATE_LIB_OK']='True'

from stable_baselines3.common.vec_env import DummyVecEnv 
from stable_baselines3 import A2C 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# For this case study please install the following libraries: gym_anytrading, yfinance, pandas_datareader, TA
import gym_anytrading
from gym_anytrading.envs import StocksEnv
import yfinance as yf
from pandas_datareader import data as pdr
# yf.pdr_override() # bug fix, for details see https://stackoverflow.com/questions/74862453/why-am-i-getting-a-typeerror-string-indices-must-be-integer-message-when-tryi
from ta import add_all_ta_features # Method from TA (Technical Analysis) library to engineer financial indicators


## Set up the S&P 500 stock index trading environment

Read some daily time series stock data

In [ ]:
df = yf.download('SPY', start='2021-01-01', end='2023-01-01', multi_level_index=False)
df.head()

In [ ]:
df = df.reset_index()
df['Date'] = pd.to_datetime(df['Date'])
df.set_index("Date", inplace = True)
df = df.reset_index()
df = df[['Date', 'Open', 'High', 'Low', 'Close', 'Volume']]
df.head()

Set up the Gym trading environment

In [31]:
env = gym.make('stocks-v0', df=df, frame_bound=(5, 400), window_size=5)

In [ ]:
env.action_space

## Execute random actions just to get familiar with the environment

In [ ]:
import time
state, info = env.reset()

# Run the environment with random actions (Buy or Sell)
while True:  # Example: run for 100 steps
    action = env.action_space.sample()  # Sample random action (0 or 1)
    next_state, reward, done, trunc, info = env.step(action)  # Take the action
    
    # Print the action and reward
    print(f"Action: {action}, Reward: {reward}, Done: {done}")
    
    if done or trunc:
        print("Episode finished. Info:", info)
        break

plt.figure(figsize=(15, 6))
plt.cla()
env.unwrapped.render_all()  # Render the environment
plt.show()

# Close the environment
env.close()

plt.show()


The green dots represent buying while the red dots represent selling stocks.

## Train an `A2C` algorithm from `stable-baselines`

In [ ]:
env = gym.make('stocks-v0', df=df, frame_bound=(5, 400), window_size=5)
model = A2C('MlpPolicy', env, verbose=1)
model.learn(total_timesteps=10000)

## Exploit the trained agent in test simulations

In [ ]:
env = gym.make('stocks-v0', df=df, frame_bound=(400, 502), window_size=5)
obs, info = env.reset()
while True:
    # obs = obs[np.newaxis, ...]
    action, states = model.predict(obs)
    obs, rewards, done,trunc,  info = env.step(action)
    
    if done or trunc:
        print(info)
        break

In [ ]:
plt.figure(figsize=(15, 6))
plt.cla()
env.unwrapped.render_all()
plt.show()

Again, the green dots represent buying while the red dots represent selling stocks. If the agent is capable of making intelligent trading decisions, it will tend to buy (green dots) when the price is relatively low, and to sell (red dots) when the price is relatively high. 

As you can see, the agent needs fine tuning to perform well. So to close the lab, let us fine tune the agent by engineering features ("financial indicators") and adding these additional indicators to the definition of the RL state, so the agent has additional information every day to make trading decisions.

## Engineer features (financial indicators) for the agent to make smarter decisions

In [ ]:
data = yf.download('SPY', start='2021-01-01', end='2023-01-01', multi_level_index=False)
# Engineer financial indicators using the method imported above from the "TA" library
df2 = add_all_ta_features(data, open='Open', high='High', low='Low', close='Close', volume='Volume', fillna=True)

In [41]:
pd.set_option('display.max_columns', None)

In [ ]:
df2.head()

In [43]:
def my_processed_data(env):
    start = env.frame_bound[0] - env.window_size
    end = env.frame_bound[1]
    prices = env.df.loc[:, 'Low'].to_numpy()[start:end]
    signal_features = env.df.loc[:, ['Close', 'Volume', 'momentum_rsi', 'volume_obv', 'trend_macd_diff']].to_numpy()[start:end]
    return prices, signal_features

class MyCustomEnv(StocksEnv):
    _process_data = my_processed_data
    

In [44]:
env2 = MyCustomEnv(df=df2, window_size= 5, frame_bound=(5, 400))

## Re-train the `A2C` algorithm from `stable-baselines` with the new engineered features

In [ ]:
model = A2C('MlpPolicy', env2, verbose=1)
model.learn(total_timesteps=10000)

## Exploit the trained agent in test simulations

In [ ]:
env = MyCustomEnv(df=df2, window_size=5, frame_bound=(400, 502))
obs, info = env.reset()

while True:
    # obs = obs[np.newaxis, ...]
    action, states = model.predict(obs)
    obs, rewards, done, trunc, info = env.step(action)
    
    if done or trunc:
        print(info)
        break

In [ ]:
plt.figure(figsize=(15, 6))
plt.cla()
env.render_all()
plt.show()

Hopefully you now see more green dots (buy stocks) when the price is relatively low, and more red dots (sell stocks) when the price is relatively high!

**Exercise 1**: What other fine tuning can you try to improve the agent's trading performance?

**Exercise 2**: Implement the `CartPole` and `SpaceInvaders` agents using the DQN algorithms from `stable-baselines`.  

**Exercise 3**: Implement the `PPO` algorithm from `stable-baselines` to learn to play at the `FlappyBird` (lab 5) and `SpaceInvaders` games.

## Thank you everyone!